In [ ]:
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

In [ ]:
from google.colab import files
  
  
uploaded = files.upload()

Saving Mental_Health_FAQ.csv to Mental_Health_FAQ.csv


In [ ]:
import pandas as pd
import io
  
df = pd.read_csv(io.BytesIO(uploaded['Mental_Health_FAQ.csv']))
print(df)

    Question_ID  ...                                            Answers
0       1590140  ...  Mental illnesses are health conditions that di...
1       2110618  ...  It is estimated that mental illness affects 1 ...
2       6361820  ...  It is estimated that mental illness affects 1 ...
3       9434130  ...  Symptoms of mental health disorders vary depen...
4       7657263  ...  When healing from mental illness, early identi...
..          ...  ...                                                ...
93      4373204  ...  Sorting out if you are drinking too much can b...
94      7807643  ...  Cannabis smoke, for example, contains cancer-c...
95      4352464  ...  You can't. But you can influence their capacit...
96      6521784  ...  Cannabidiol or CBD is a naturally occurring co...
97      3221856  ...  "Vaping" is the term for using a device where ...

[98 rows x 3 columns]


In [ ]:
df.shape[0] # returns the number of rows in dataset

98

In [ ]:
df.ffill(axis = 0,inplace=True) # fills the null value with the previous value.
df

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...,...
93,4373204,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,7807643,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,4352464,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,6521784,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [ ]:
df1=df.head(10) # copy of first ten rows of dataset

In [ ]:

# function that converts text into lower case and removes special characters

def step1(x):
    for i in x:
        a=str(i).lower()
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

In [ ]:
step1(df1['Questions'])

what does it mean to have a mental illness 
who does mental illness affect 
what causes mental illness 
what are some of the warning signs of mental illness 
can people with mental illness recover 
what should i do if i know someone who appears to have the symptoms of a mental disorder 
how can i find a mental health professional for myself or my child 
what treatment options are available 
if i become involved in treatment  what do i need to know 
what is the difference between mental health professionals 


In [ ]:
nltk.download('punkt')
# word tokenizing
    
s='tell me about your personality'
words=word_tokenize(s)
print(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['tell', 'me', 'about', 'your', 'personality']


In [ ]:
nltk.download('wordnet')
lemma = wordnet.WordNetLemmatizer() # intializing lemmatizer
lemma.lemmatize('absorbed', pos = 'v')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


'absorb'

In [ ]:
nltk.download('averaged_perceptron_tagger')
pos_tag(nltk.word_tokenize(s),tagset = None) # returns the parts of speech of every word

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('tell', 'VB'),
 ('me', 'PRP'),
 ('about', 'IN'),
 ('your', 'PRP$'),
 ('personality', 'NN')]

In [ ]:

# function that performs text normalization steps

def text_normalization(text):
    text=str(text).lower() # text to lower case
    spl_char_text=re.sub(r'[^ a-z]','',text) # removing special characters
    tokens=nltk.word_tokenize(spl_char_text) # word tokenizing
    lema=wordnet.WordNetLemmatizer() # intializing lemmatization
    tags_list=pos_tag(tokens,tagset=None) # parts of speech
    lema_words=[]   # empty list 
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adjective
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lema.lemmatize(token,pos_val) # performing lemmatization
        lema_words.append(lema_token) # appending the lemmatized token into a list
    
    return " ".join(lema_words) # returns the lemmatized tokens as a sentence

In [ ]:
text_normalization('telling you some stuff about me')

'tell you some stuff about me'

In [ ]:
df['lemmatized_text']=df['Questions'].apply(text_normalization) # applying the fuction to the dataset to get clean text
df.tail(15)

,Question_ID,Questions,Answers,lemmatized_text
83,9679704,What is antisocial personality disorder?,A personality disorder is a pattern of thought...,what be antisocial personality disorder
84,9167296,What is obsessive-compulsive personality disor...,A personality disorder is a pattern of thought...,what be obsessivecompulsive personality disorder
85,9049618,What is binge-eating disorder?,Binge-eating disorder or BED is a type of eati...,what be bingeeating disorder
86,7984793,What’s the difference between dissociative ide...,"Sometimes, people confuse dissociative identit...",whats the difference between dissociative iden...
87,6085633,What's the difference between psychosis and sc...,Psychosis is a syndrome or group of symptoms. ...,whats the difference between psychosis and sch...
88,1896541,What are positive and negative symptoms?,Positive and negative symptoms are medical ter...,what be positive and negative symptom
89,4962901,What is a prodrome?,Prodrome is a medical term for early signs or ...,what be a prodrome
90,6869967,I’m an adult and I think I might have ADHD. Wh...,While we think of attention-deficit/hyperactiv...,im an adult and i think i might have adhd what...
91,4824231,What’s the difference between substance use an...,People tend to use the word “addiction” to mea...,whats the difference between substance use and...
92,4294616,How can I find help for an alcohol or drug use...,Seeking to make changes in the way you use sub...,how can i find help for an alcohol or drug use...


In [ ]:
nltk.download('stopwords')
# all the stop words we have 

stop = stopwords.words('english')
print(stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
cv = CountVectorizer() # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [ ]:
# returns all the unique word from data 

features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()


,about,addiction,adhd,adult,advance,affect,after,age,alcohol,allow,an,and,antidepressant,antisocial,anxiety,any,appear,apply,assistance,available,bad,balanced,be,become,before,between,bingeeating,borderline,brain,but,can,cannabis,care,cause,cbd,cbt,certain,challenge,child,clinical,...,support,symptom,take,talk,teenager,tell,the,therapy,there,think,thinking,this,thought,time,to,too,trap,treatment,trial,type,unwell,use,vaping,warning,we,well,what,whats,when,where,who,why,with,wont,work,worried,worry,you,young,youth
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
Question ='What treatment options are available' # considering an example query

In [ ]:
# checking for stop words

Q=[]
a=Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b=" ".join(Q)

In [ ]:
Question_lemma = text_normalization(b) # applying the function that we created for text normalizing
Question_bow = cv.transform([Question_lemma]).toarray() # applying bow

In [ ]:
text_normalization

<function __main__.text_normalization>

In [ ]:
Question_bow

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:

# cosine similarity for the above question we considered.

cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.1767767 ],
       [0.        ],
       [0.25      ],
       [0.14433757],
       [0.        ],
       [0.12909944],
       [0.        ],
       [0.89442719],
       [0.31622777],
       [0.1767767 ],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.18898224],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.16666667],
       [0.16666667],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.25      ],
       [0.15811388],
       [0.        ],
       [0.        ],
       [0.2236068 ],
       [0.14433757],
       [0.        ],
       [0.15811388],
       [0.14433757],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.16666667],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [ ]:

df['similarity_bow']=cosine_value # creating a new column

In [ ]:
df_simi = pd.DataFrame(df, columns=['Answers','similarity_bow']) # taking similarity value of responses for the question we took
df_simi


,Answers,similarity_bow
0,Mental illnesses are health conditions that di...,0.176777
1,It is estimated that mental illness affects 1 ...,0.000000
2,It is estimated that mental illness affects 1 ...,0.250000
3,Symptoms of mental health disorders vary depen...,0.144338
4,"When healing from mental illness, early identi...",0.000000
...,...,...
93,Sorting out if you are drinking too much can b...,0.000000
94,"Cannabis smoke, for example, contains cancer-c...",0.000000
95,You can't. But you can influence their capacit...,0.000000
96,Cannabidiol or CBD is a naturally occurring co...,0.158114


In [ ]:

df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False) # sorting the values
df_simi_sort.head()

,Answers,similarity_bow
7,Just as there are different types of medicatio...,0.894427
8,Since beginning treatment is a big step for in...,0.316228
11,Beginning treatment is a big step for individu...,0.316228
89,Prodrome is a medical term for early signs or ...,0.288675
57,MSP stands for Medical Services Plan. It’s a h...,0.288675


In [ ]:
threshold = 0.1 # considering the value of p=smiliarity to be greater than 0.1
df_threshold = df_simi_sort[df_simi_sort['similarity_bow'] > threshold] 
df_threshold

,Answers,similarity_bow
7,Just as there are different types of medicatio...,0.894427
8,Since beginning treatment is a big step for in...,0.316228
11,Beginning treatment is a big step for individu...,0.316228
89,Prodrome is a medical term for early signs or ...,0.288675
57,MSP stands for Medical Services Plan. It’s a h...,0.288675
58,A referral means someone recommends you to ano...,0.288675
2,It is estimated that mental illness affects 1 ...,0.250000
85,Binge-eating disorder or BED is a type of eati...,0.250000
24,We all have mental health which is made up of ...,0.250000
79,Cyclothymic disorder is a subtype of bipolar d...,0.250000


In [ ]:
index_value = cosine_value.argmax() # returns the index number of highest value
index_value

7

In [ ]:
(Question)

'What treatment options are available'

In [ ]:

df['Answers'].loc[index_value] # The text at the above index becomes the response for the question

'Just as there are different types of medications for physical illness, different treatment options are available for individuals with mental illness. Treatment works differently for different people. It is important to find what works best for you or your child.'

In [ ]:
Question1 ='What treatment options are available'

In [ ]:

# using tf-idf

tfidf=TfidfVectorizer() # intializing tf-id 
x_tfidf=tfidf.fit_transform(df['lemmatized_text']).toarray() # transforming the data into array

In [ ]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray() # applying tf-idf


In [ ]:
# returns all the unique word from data with a score of that word

df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names()) 
df_tfidf.head()

,about,addiction,adhd,adult,advance,affect,after,age,alcohol,allow,an,and,antidepressant,antisocial,anxiety,any,appear,apply,assistance,available,bad,balanced,be,become,before,between,bingeeating,borderline,brain,but,can,cannabis,care,cause,cbd,cbt,certain,challenge,child,clinical,...,support,symptom,take,talk,teenager,tell,the,therapy,there,think,thinking,this,thought,time,to,too,trap,treatment,trial,type,unwell,use,vaping,warning,we,well,what,whats,when,where,who,why,with,wont,work,worried,worry,you,young,youth
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.332326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.220070,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.590436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.541598,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.688269,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.299635,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.210416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.396806,0.0,0.0,0.158459,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.185783,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.414843,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

cos=1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')  # applying cosine similarity
cos

array([[0.0504562 ],
       [0.        ],
       [0.0686984 ],
       [0.08488045],
       [0.        ],
       [0.03358704],
       [0.        ],
       [1.        ],
       [0.20208366],
       [0.13154037],
       [0.        ],
       [0.20208366],
       [0.        ],
       [0.04128706],
       [0.03508416],
       [0.        ],
       [0.        ],
       [0.18657351],
       [0.10435026],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.22086907],
       [0.04628936],
       [0.        ],
       [0.0665589 ],
       [0.06287489],
       [0.04433521],
       [0.        ],
       [0.03909618],
       [0.03848933],
       [0.        ],
       [0.05531838],
       [0.        ],
       [0.03828114],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [ ]:
df['similarity_tfidf']=cos # creating a new column 
df_simi_tfidf = pd.DataFrame(df, columns=['Answers','similarity_tfidf']) # taking similarity value of responses for the question we took
df_simi_tfidf

,Answers,similarity_tfidf
0,Mental illnesses are health conditions that di...,0.050456
1,It is estimated that mental illness affects 1 ...,0.000000
2,It is estimated that mental illness affects 1 ...,0.068698
3,Symptoms of mental health disorders vary depen...,0.084880
4,"When healing from mental illness, early identi...",0.000000
...,...,...
93,Sorting out if you are drinking too much can b...,0.000000
94,"Cannabis smoke, for example, contains cancer-c...",0.097116
95,You can't. But you can influence their capacit...,0.000000
96,Cannabidiol or CBD is a naturally occurring co...,0.085588


In [ ]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False) # sorting the values
df_simi_tfidf_sort.head(10)

,Answers,similarity_tfidf
7,Just as there are different types of medicatio...,1.000000
24,We all have mental health which is made up of ...,0.220869
8,Since beginning treatment is a big step for in...,0.202084
11,Beginning treatment is a big step for individu...,0.202084
17,Mental health conditions are often treated wit...,0.186574
57,MSP stands for Medical Services Plan. It’s a h...,0.182583
89,Prodrome is a medical term for early signs or ...,0.182583
58,A referral means someone recommends you to ano...,0.182583
79,Cyclothymic disorder is a subtype of bipolar d...,0.159528
85,Binge-eating disorder or BED is a type of eati...,0.159528


In [ ]:

threshold = 0.1 # considering the value of p=smiliarity to be greater than 0.1
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold] 
df_threshold

,Answers,similarity_tfidf
7,Just as there are different types of medicatio...,1.000000
24,We all have mental health which is made up of ...,0.220869
8,Since beginning treatment is a big step for in...,0.202084
11,Beginning treatment is a big step for individu...,0.202084
17,Mental health conditions are often treated wit...,0.186574
57,MSP stands for Medical Services Plan. It’s a h...,0.182583
89,Prodrome is a medical term for early signs or ...,0.182583
58,A referral means someone recommends you to ano...,0.182583
79,Cyclothymic disorder is a subtype of bipolar d...,0.159528
85,Binge-eating disorder or BED is a type of eati...,0.159528


In [ ]:

index_value1 = cos.argmax() # returns the index number of highest value
index_value1

7

In [ ]:

Question1

'What treatment options are available'

In [ ]:
df['Answers'].loc[index_value1]  # returns the text at that index

'Just as there are different types of medications for physical illness, different treatment options are available for individuals with mental illness. Treatment works differently for different people. It is important to find what works best for you or your child.'

In [ ]:
# Function that removes stop words and process the text

def stopword_(text):   
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
    stop=stopwords.words('english')
    lema=wordnet.WordNetLemmatizer()
    lema_word=[]
    for token,pos_token in tag_list:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
        lema_word.append(lema_token)
    return " ".join(lema_word)

In [ ]:
# defining a function that returns response to query using bow

def chat_bow(text):
    s=stopword_(text)
    lemma=text_normalization(s) # calling the function to perform text normalization
    bow=cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value=cosine_value.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [ ]:

chat_bow('can you prevent mental health problems')

'We can all suffer from mental health challenges, but developing our wellbeing, resilience, and seeking help early can help prevent challenges becoming serious.'

In [ ]:
chat_bow('what is mental health')

'We all have mental health which is made up of our beliefs, thoughts, feelings and behaviours.'

In [ ]:
chat_bow('are there cures for mental health problems')

'It is often more realistic and helpful to find out what helps with the issues you face. Talking, counselling, medication, friendships, exercise, good sleep and nutrition, and meaningful occupation can all help.'

In [ ]:
chat_bow('how do I know if i am unwell')

'If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.'

In [ ]:
chat_bow('what do you mean by mental health')

'We all have mental health which is made up of our beliefs, thoughts, feelings and behaviours.'

In [ ]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma=text_normalization(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [ ]:
chat_tfidf('what is mental health')

'We all have mental health which is made up of our beliefs, thoughts, feelings and behaviours.'

In [ ]:
chat_tfidf('what is vaping')

'"Vaping" is the term for using a device where liquids, often flavoured, are turned into vapour (hence, vaping) and inhaled. A vaping device consists of a mouthpiece, tank or reservoir to hold the liquid, a heating element and a battery to power the element. The liquid is heated to become a mist, inhaled through the mouth into the lungs where the nicotine or cannabis enters the bloodstream. Any residual vapour is exhaled. Vaping devices come in several types, shapes, and sizes, referred to as e-cigarettes, vape pens, vapes, mods, tanks, or e-hookahs. Vaping liquid (e-liquid, e-juice) consists of a solvent, usually propylene glycol or glycerol, flavour, and nicotine or cannabis, though it may not contain anything. In British Columbia you must be 19 years old to buy vaping products. \n The number of young people who smoke cigarettes has declined over the past few years, while vaping has increased. E-cigarettes were developed as an alternative to smoking. The first devices were available 

In [ ]:
chat_tfidf('how do i see a counsellor')

'You can find directories of counsellors through their professional organizations. \n Registered Clinical Counsellors: visit the BC Association of Clinical Counsellors \n Canadian Certified Counsellors: visit the Canadian Counselling and Psychotherapy Association \n Canadian Professional Counsellors: visit the Canadian Professional Counsellors Association \n For more specialized counselling professionals, such as Marriage and Family Therapists, see the Federation of Associations for Counselling Therapists in British Columbia. \n You can make an appointment on your own. You don’t need a doctor’s referral to see a counsellor. \n Counsellors are not usually covered by MSP, so you’ll have to pay for the cost of the appointment yourself. Workplace extended health benefits, Employee (Family) Assistance Programs, or private health insurance plans may help cover some costs. Some counsellors may offer a sliding scale based on your income. If you’re facing financial hardship, you can ask when yo

In [ ]:
chat_tfidf('how to find a support group')

'Support groups are a way for people with a common experience to help each other and learn from each other. There are support groups for people with any experience of mental illness, support groups for people with a specific diagnosis, support groups for family members and friends, and more. \n Support groups are offered by community organizations, mental health service providers, schools, campuses, and support agencies. Here are some general resources to help you find a mental health or substance use related support group in BC: \n Talk to your mental health care provider. They might be able to make recommendations. \n The Mood Disorders Association of BC offers a network of peer support groups around the province for people who experience mental illnesses like mood disorders or post-traumatic stress disorder. They also have groups for family and friends. Groups are drop-in. \n Many Canadian Mental Health Association branches offer support services, including support groups. If your l